<a href="https://colab.research.google.com/github/sunshineluyao/ESG_TwoSources/blob/main/ESG_Abnormal_Return_Ref.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Creat the Leader, Average, and Laggard Group

In [2]:
import pandas as pd
import numpy as np

In [3]:
df =pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/Benchmark.csv',index_col='Unnamed: 0')
df.head()

,Month,Ticker,Company_name,Stock_exchange,Industry,SIC_Code,BV_Y1,BV_Y2,BV_Y3,BV_Y4,BV_Y5,MV_Y1,MV_Y2,MV_Y3,MV_Y4,MV_Y5,BookValue_Quarterly,MarketValue_Quarterly,Size,Momentum,Monthly_Return,Longterm_BV,Longterm_MV,Longterm_BMV,SIC_2,Industry_Ave_BMR,BMR_Quarterly,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Portfoio_Return
0,1,A,Agilent Technologies Inc,NYSE,Healthcare,3826,4243.000,4831.000,4567.000,4748.000,4873.000,13888.06491,14658.83346,21632.51727,21488.23982,26461.74713,4848.000,26461.74713,26461.71,11.637870,-3.223538,4652.4000,19625.880518,0.237054,38.0,0.293538,0.183208,-0.110330,5,3.0,2.0,5.32,-1.998444
1,1,EMR,Emerson Electric Co,NYSE,Industrials,3823,7568.000,8718.000,8947.000,8233.000,8405.000,31154.84501,35928.08990,44728.42451,37412.97623,46454.07146,8421.000,46454.07146,46588.23,9.389818,-6.071341,8374.2000,39135.681422,0.213979,38.0,0.293538,0.181276,-0.112262,5,3.0,2.0,5.32,-1.998444
2,1,ISRG,Intuitive Surgical Inc,NASDAQ,Healthcare,3842,4319.500,5777.800,4778.800,6678.800,8263.800,20407.38309,24572.19197,40890.73727,54692.79043,68321.27748,8263.800,68321.27748,68321.57,11.684600,-5.306607,5963.7400,41776.876048,0.142752,38.0,0.293538,0.120955,-0.172583,5,3.0,2.0,5.32,-1.998444
3,1,LUV,Southwest Airlines Co,NYSE,Industrials,4512,7358.000,8441.000,9641.000,9853.000,9832.000,28004.29095,30681.39126,38837.22235,26136.01040,28408.38528,9832.000,28408.38528,28408.38,5.548429,1.852536,9025.0000,30413.460048,0.296744,45.0,0.541633,0.346095,-0.195538,5,3.0,2.0,5.32,-1.998444
4,1,RCL,Royal Caribbean Cruises Ltd,NYSE,Consumer Cyclicals,4481,8063.039,9121.412,10702.303,11105.461,12163.846,22265.73352,17604.30925,25536.52996,20376.32002,27991.79939,12163.846,27991.79939,27987.83,6.146634,-12.306190,10231.2122,22754.938428,0.449626,45.0,0.541633,0.434550,-0.107083,5,3.0,2.0,5.32,-1.998444


In [4]:
ESG = pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/ESG_Ref1226.csv')
ESG=ESG[['Month','Ticker','ESG_Rating','ESG_Score']]
ESG.head()

,Month,Ticker,ESG_Rating,ESG_Score
0,1,A,A,86.807273
1,1,AAL,B+,66.734934
2,1,AAOI,C-,25.381834
3,1,AAON,D+,22.007720
4,1,AAP,C+,44.134254


### Categorize by ESG_Score

In [5]:
Month_List = ESG['Month'].unique().tolist()
Month_List

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [6]:
df_ =pd.DataFrame()
for i in Month_List:
  df_temp = ESG[ESG.Month==i]
  df_temp['ESG_G']=pd.qcut(df_temp['ESG_Score'],5,labels=np.arange(1,6,1))
  df_ =df_.append(df_temp)
df_.tail()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

,Month,Ticker,ESG_Rating,ESG_Score,ESG_G
16231,11,ZEUS,C-,29.698024,2
16232,11,ZION,C+,46.009323,3
16233,11,ZTS,B-,55.372768,4
16234,11,ZUMZ,C-,30.361351,2
16235,11,ZYXI,D+,24.951316,1


In [7]:
ESG =df_
ESG.head()

,Month,Ticker,ESG_Rating,ESG_Score,ESG_G
0,1,A,A,86.807273,5
1,1,AAL,B+,66.734934,5
2,1,AAOI,C-,25.381834,1
3,1,AAON,D+,22.007720,1
4,1,AAP,C+,44.134254,3


In [8]:
df=df.merge(ESG, on=['Month','Ticker'])
df.head()

,Month,Ticker,Company_name,Stock_exchange,Industry,SIC_Code,BV_Y1,BV_Y2,BV_Y3,BV_Y4,BV_Y5,MV_Y1,MV_Y2,MV_Y3,MV_Y4,MV_Y5,BookValue_Quarterly,MarketValue_Quarterly,Size,Momentum,Monthly_Return,Longterm_BV,Longterm_MV,Longterm_BMV,SIC_2,Industry_Ave_BMR,BMR_Quarterly,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Portfoio_Return,ESG_Rating,ESG_Score,ESG_G
0,1,A,Agilent Technologies Inc,NYSE,Healthcare,3826,4243.000,4831.000,4567.000,4748.000,4873.000,13888.06491,14658.83346,21632.51727,21488.23982,26461.74713,4848.000,26461.74713,26461.71,11.637870,-3.223538,4652.4000,19625.880518,0.237054,38.0,0.293538,0.183208,-0.110330,5,3.0,2.0,5.32,-1.998444,A,86.807273,5
1,1,EMR,Emerson Electric Co,NYSE,Industrials,3823,7568.000,8718.000,8947.000,8233.000,8405.000,31154.84501,35928.08990,44728.42451,37412.97623,46454.07146,8421.000,46454.07146,46588.23,9.389818,-6.071341,8374.2000,39135.681422,0.213979,38.0,0.293538,0.181276,-0.112262,5,3.0,2.0,5.32,-1.998444,B+,67.034875,5
2,1,ISRG,Intuitive Surgical Inc,NASDAQ,Healthcare,3842,4319.500,5777.800,4778.800,6678.800,8263.800,20407.38309,24572.19197,40890.73727,54692.79043,68321.27748,8263.800,68321.27748,68321.57,11.684600,-5.306607,5963.7400,41776.876048,0.142752,38.0,0.293538,0.120955,-0.172583,5,3.0,2.0,5.32,-1.998444,C+,48.686488,3
3,1,LUV,Southwest Airlines Co,NYSE,Industrials,4512,7358.000,8441.000,9641.000,9853.000,9832.000,28004.29095,30681.39126,38837.22235,26136.01040,28408.38528,9832.000,28408.38528,28408.38,5.548429,1.852536,9025.0000,30413.460048,0.296744,45.0,0.541633,0.346095,-0.195538,5,3.0,2.0,5.32,-1.998444,B+,69.810635,5
4,1,RCL,Royal Caribbean Cruises Ltd,NYSE,Consumer Cyclicals,4481,8063.039,9121.412,10702.303,11105.461,12163.846,22265.73352,17604.30925,25536.52996,20376.32002,27991.79939,12163.846,27991.79939,27987.83,6.146634,-12.306190,10231.2122,22754.938428,0.449626,45.0,0.541633,0.434550,-0.107083,5,3.0,2.0,5.32,-1.998444,B+,71.869878,5


In [9]:
df['ESG_Group']=df['ESG_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
df.head()

,Month,Ticker,Company_name,Stock_exchange,Industry,SIC_Code,BV_Y1,BV_Y2,BV_Y3,BV_Y4,BV_Y5,MV_Y1,MV_Y2,MV_Y3,MV_Y4,MV_Y5,BookValue_Quarterly,MarketValue_Quarterly,Size,Momentum,Monthly_Return,Longterm_BV,Longterm_MV,Longterm_BMV,SIC_2,Industry_Ave_BMR,BMR_Quarterly,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Portfoio_Return,ESG_Rating,ESG_Score,ESG_G,ESG_Group
0,1,A,Agilent Technologies Inc,NYSE,Healthcare,3826,4243.000,4831.000,4567.000,4748.000,4873.000,13888.06491,14658.83346,21632.51727,21488.23982,26461.74713,4848.000,26461.74713,26461.71,11.637870,-3.223538,4652.4000,19625.880518,0.237054,38.0,0.293538,0.183208,-0.110330,5,3.0,2.0,5.32,-1.998444,A,86.807273,5,Leader
1,1,EMR,Emerson Electric Co,NYSE,Industrials,3823,7568.000,8718.000,8947.000,8233.000,8405.000,31154.84501,35928.08990,44728.42451,37412.97623,46454.07146,8421.000,46454.07146,46588.23,9.389818,-6.071341,8374.2000,39135.681422,0.213979,38.0,0.293538,0.181276,-0.112262,5,3.0,2.0,5.32,-1.998444,B+,67.034875,5,Leader
2,1,ISRG,Intuitive Surgical Inc,NASDAQ,Healthcare,3842,4319.500,5777.800,4778.800,6678.800,8263.800,20407.38309,24572.19197,40890.73727,54692.79043,68321.27748,8263.800,68321.27748,68321.57,11.684600,-5.306607,5963.7400,41776.876048,0.142752,38.0,0.293538,0.120955,-0.172583,5,3.0,2.0,5.32,-1.998444,C+,48.686488,3,Average
3,1,LUV,Southwest Airlines Co,NYSE,Industrials,4512,7358.000,8441.000,9641.000,9853.000,9832.000,28004.29095,30681.39126,38837.22235,26136.01040,28408.38528,9832.000,28408.38528,28408.38,5.548429,1.852536,9025.0000,30413.460048,0.296744,45.0,0.541633,0.346095,-0.195538,5,3.0,2.0,5.32,-1.998444,B+,69.810635,5,Leader
4,1,RCL,Royal Caribbean Cruises Ltd,NYSE,Consumer Cyclicals,4481,8063.039,9121.412,10702.303,11105.461,12163.846,22265.73352,17604.30925,25536.52996,20376.32002,27991.79939,12163.846,27991.79939,27987.83,6.146634,-12.306190,10231.2122,22754.938428,0.449626,45.0,0.541633,0.434550,-0.107083,5,3.0,2.0,5.32,-1.998444,B+,71.869878,5,Leader


# Step 2: Calculating ESG Portfolio Return Controlling Benchmark Group

In [10]:
temp = df.groupby(by=['Month','Benchmark','ESG_Group'],as_index=False)['Monthly_Return'].mean()
temp.columns=['Month','Benchmark','ESG_Group','ESG_Portfoio_Return']
temp.tail()

,Month,Benchmark,ESG_Group,ESG_Portfoio_Return
3032,11,5.54,Laggard,20.645230
3033,11,5.54,Leader,14.218714
3034,11,5.55,Average,10.951482
3035,11,5.55,Laggard,23.741600
3036,11,5.55,Leader,11.935651


In [11]:
df=df.merge(temp,on=['Month','Benchmark','ESG_Group'])
df.head()

,Month,Ticker,Company_name,Stock_exchange,Industry,SIC_Code,BV_Y1,BV_Y2,BV_Y3,BV_Y4,BV_Y5,MV_Y1,MV_Y2,MV_Y3,MV_Y4,MV_Y5,BookValue_Quarterly,MarketValue_Quarterly,Size,Momentum,Monthly_Return,Longterm_BV,Longterm_MV,Longterm_BMV,SIC_2,Industry_Ave_BMR,BMR_Quarterly,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Portfoio_Return,ESG_Rating,ESG_Score,ESG_G,ESG_Group,ESG_Portfoio_Return
0,1,A,Agilent Technologies Inc,NYSE,Healthcare,3826,4243.000,4831.000,4567.000,4748.000,4873.000,13888.06491,14658.83346,21632.51727,21488.23982,26461.74713,4848.000,26461.74713,26461.71,11.637870,-3.223538,4652.4000,19625.880518,0.237054,38.0,0.293538,0.183208,-0.110330,5,3.0,2.0,5.32,-1.998444,A,86.807273,5,Leader,-1.144085
1,1,EMR,Emerson Electric Co,NYSE,Industrials,3823,7568.000,8718.000,8947.000,8233.000,8405.000,31154.84501,35928.08990,44728.42451,37412.97623,46454.07146,8421.000,46454.07146,46588.23,9.389818,-6.071341,8374.2000,39135.681422,0.213979,38.0,0.293538,0.181276,-0.112262,5,3.0,2.0,5.32,-1.998444,B+,67.034875,5,Leader,-1.144085
2,1,LUV,Southwest Airlines Co,NYSE,Industrials,4512,7358.000,8441.000,9641.000,9853.000,9832.000,28004.29095,30681.39126,38837.22235,26136.01040,28408.38528,9832.000,28408.38528,28408.38,5.548429,1.852536,9025.0000,30413.460048,0.296744,45.0,0.541633,0.346095,-0.195538,5,3.0,2.0,5.32,-1.998444,B+,69.810635,5,Leader,-1.144085
3,1,RCL,Royal Caribbean Cruises Ltd,NYSE,Consumer Cyclicals,4481,8063.039,9121.412,10702.303,11105.461,12163.846,22265.73352,17604.30925,25536.52996,20376.32002,27991.79939,12163.846,27991.79939,27987.83,6.146634,-12.306190,10231.2122,22754.938428,0.449626,45.0,0.541633,0.434550,-0.107083,5,3.0,2.0,5.32,-1.998444,B+,71.869878,5,Leader,-1.144085
4,1,MRK,Merck & Co Inc,NYSE,Healthcare,2834,44676.000,40088.000,34336.000,26701.000,25907.000,147554.94850,162312.68560,153304.06070,198694.76840,231557.25770,25907.000,231557.25770,231557.20,9.881533,-6.058270,34341.6000,178684.744180,0.192191,28.0,0.278208,0.111882,-0.166326,5,3.0,2.0,5.32,-1.998444,A-,81.033146,5,Leader,-1.144085


# Step 3: Calculating Abonormal Return

In [12]:
df['ESG_Abnormal_Return']=df['ESG_Portfoio_Return']-df['Portfoio_Return']
df.head()

,Month,Ticker,Company_name,Stock_exchange,Industry,SIC_Code,BV_Y1,BV_Y2,BV_Y3,BV_Y4,BV_Y5,MV_Y1,MV_Y2,MV_Y3,MV_Y4,MV_Y5,BookValue_Quarterly,MarketValue_Quarterly,Size,Momentum,Monthly_Return,Longterm_BV,Longterm_MV,Longterm_BMV,SIC_2,Industry_Ave_BMR,BMR_Quarterly,Industry_Adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Benchmark,Portfoio_Return,ESG_Rating,ESG_Score,ESG_G,ESG_Group,ESG_Portfoio_Return,ESG_Abnormal_Return
0,1,A,Agilent Technologies Inc,NYSE,Healthcare,3826,4243.000,4831.000,4567.000,4748.000,4873.000,13888.06491,14658.83346,21632.51727,21488.23982,26461.74713,4848.000,26461.74713,26461.71,11.637870,-3.223538,4652.4000,19625.880518,0.237054,38.0,0.293538,0.183208,-0.110330,5,3.0,2.0,5.32,-1.998444,A,86.807273,5,Leader,-1.144085,0.854359
1,1,EMR,Emerson Electric Co,NYSE,Industrials,3823,7568.000,8718.000,8947.000,8233.000,8405.000,31154.84501,35928.08990,44728.42451,37412.97623,46454.07146,8421.000,46454.07146,46588.23,9.389818,-6.071341,8374.2000,39135.681422,0.213979,38.0,0.293538,0.181276,-0.112262,5,3.0,2.0,5.32,-1.998444,B+,67.034875,5,Leader,-1.144085,0.854359
2,1,LUV,Southwest Airlines Co,NYSE,Industrials,4512,7358.000,8441.000,9641.000,9853.000,9832.000,28004.29095,30681.39126,38837.22235,26136.01040,28408.38528,9832.000,28408.38528,28408.38,5.548429,1.852536,9025.0000,30413.460048,0.296744,45.0,0.541633,0.346095,-0.195538,5,3.0,2.0,5.32,-1.998444,B+,69.810635,5,Leader,-1.144085,0.854359
3,1,RCL,Royal Caribbean Cruises Ltd,NYSE,Consumer Cyclicals,4481,8063.039,9121.412,10702.303,11105.461,12163.846,22265.73352,17604.30925,25536.52996,20376.32002,27991.79939,12163.846,27991.79939,27987.83,6.146634,-12.306190,10231.2122,22754.938428,0.449626,45.0,0.541633,0.434550,-0.107083,5,3.0,2.0,5.32,-1.998444,B+,71.869878,5,Leader,-1.144085,0.854359
4,1,MRK,Merck & Co Inc,NYSE,Healthcare,2834,44676.000,40088.000,34336.000,26701.000,25907.000,147554.94850,162312.68560,153304.06070,198694.76840,231557.25770,25907.000,231557.25770,231557.20,9.881533,-6.058270,34341.6000,178684.744180,0.192191,28.0,0.278208,0.111882,-0.166326,5,3.0,2.0,5.32,-1.998444,A-,81.033146,5,Leader,-1.144085,0.854359


# Step 4: Visualizing Abnormal Return Monthly

In [13]:
import plotly.express as px
fig = px.box(df, x="ESG_Group", y="ESG_Abnormal_Return",color='ESG_Group',animation_frame='Month',animation_group='Ticker')
fig.show()

In [14]:
ESG_summary = df.groupby(by=['Month','ESG_Group'],as_index=False)['ESG_Abnormal_Return'].agg(['mean','std','count']).reset_index()
ESG_summary['sqrt_count']=ESG_summary['count'].apply(lambda x: np.sqrt(x))
ESG_summary['mean_std']=ESG_summary['std']/ESG_summary['sqrt_count']
ESG_summary.head()

,Month,ESG_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,0.070894,1.903787,843,29.034462,0.065570
1,1,Laggard,0.208548,4.787836,280,16.733201,0.286128
2,1,Leader,-0.406039,3.477502,291,17.058722,0.203855
3,2,Average,-0.128619,2.208611,843,29.034462,0.076069
4,2,Laggard,0.450232,5.008812,280,16.733201,0.299334


In [15]:
import plotly.express as px
fig = px.line(ESG_summary, x="Month", y="mean", color="ESG_Group", error_y="mean_std")
fig.show()

In [16]:
ESG_summary = ESG_summary[ESG_summary['ESG_Group'].isin(['Leader', 'Laggard'])]

In [17]:
import plotly.express as px
fig = px.line(ESG_summary, x="Month", y="mean", color="ESG_Group", error_y="mean_std")
fig.show()

# Step 5: Conduct Two-group t-test for each month

In [18]:
pip install pingouin

     |████████████████████████████████| 225kB 11.8MB/s 
  Created wheel for pingouin: filename=pingouin-0.3.8-cp36-none-any.whl size=221687 sha256=ea958a842565753e2622122e121c87c0a482a59e1f0ab8c5ffcbe95745d11bbd
  Stored in directory: /root/.cache/pip/wheels/d6/9e/53/f885f73f29cf7c8cac3d8f4b1532bbfef2f5eb543946ac9055
  Created wheel for outdated: filename=outdated-0.2.0-cp36-none-any.whl size=4961 sha256=64ada042abd6893515cf0ff15ecc4c235b92f46f875c56918a2725a9a9face90
  Stored in directory: /root/.cache/pip/wheels/fd/7c/ef/814f514d31197310872b5abf353feb8fef9d67ee658e1e7e39
  Created wheel for littleutils: filename=littleutils-0.2.2-cp36-none-any.whl size=7051 sha256=ba2267c149fe029ed1c28d9ca66d4042973156c5d6c577ce116a8a5f6c0d0e09
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built pingouin outdated littleutils


In [19]:
import pingouin as pg


Two-Sample T-Test

https://pingouin-stats.org/generated/pingouin.ttest.html#pingouin.ttest

In [20]:
temp.head()

,Month,Benchmark,ESG_Group,ESG_Portfoio_Return
0,1,1.11,Average,-0.427648
1,1,1.11,Laggard,-11.817850
2,1,1.11,Leader,-0.678885
3,1,1.12,Average,1.086716
4,1,1.12,Laggard,-3.378531


In [21]:
temp=temp.pivot(index=['Month','Benchmark'], columns='ESG_Group', values='ESG_Portfoio_Return').reset_index()
temp.head()

ESG_Group,Month,Benchmark,Average,Laggard,Leader
0,1,1.11,-0.427648,-11.817850,-0.678885
1,1,1.12,1.086716,-3.378531,NaN
2,1,1.13,-6.563007,-2.208887,NaN
3,1,1.14,-5.531477,2.098884,NaN
4,1,1.15,2.606219,3.643520,NaN


### Test Monthly the Leaders' Abornormal Return >0?

In [22]:
temp.isna().sum()

ESG_Group
Month          0
Benchmark      0
Average        3
Laggard      280
Leader       430
dtype: int64

In [23]:
# temp=temp.dropna()

In [24]:
Leader = temp[temp['Month']==1]['Leader']
t_test = pg.ttest(Leader, 0)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,-5.130672,83,two-sided,0.000002,"[-4.75, -2.09]",0.559802,8059.031,0.999066


In [25]:
df_ = pd.DataFrame()
Month_List =temp.Month.unique().tolist()
for i in Month_List:
  Leader = temp[temp.Month==i]['Leader']
  t_test = pg.ttest(Leader, 0)
  t_test['Month']=i
  df_=df_.append(t_test)

In [26]:
df_

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power,Month
T-test,-5.130672,83,two-sided,1.864207e-06,"[-4.75, -2.09]",0.559802,8059.031,0.999066,1
T-test,-16.369586,82,two-sided,1.450631e-27,"[-10.27, -8.05]",1.796796,2.403e+24,1.000000,2
T-test,-14.610280,82,two-sided,1.542555e-24,"[-27.6, -20.99]",1.603687,2.594e+21,1.000000,3
T-test,15.221202,80,two-sided,2.435573e-25,"[21.21, 27.59]",1.691245,1.579e+22,1.000000,4
T-test,8.840831,79,two-sided,1.990187e-13,"[6.07, 9.6]",0.988435,3.857e+10,1.000000,5
T-test,2.421885,84,two-sided,1.759187e-02,"[0.34, 3.47]",0.262690,1.872,0.667919,6
T-test,4.745630,82,two-sided,8.687209e-06,"[2.29, 5.6]",0.520901,1905.815,0.996828,7
T-test,-4.569565,82,two-sided,1.707073e-05,"[-4.78, -1.88]",0.501575,1013.598,0.994698,9
T-test,0.289618,77,two-sided,7.728869e-01,"[-1.42, 1.91]",0.032793,0.13,0.059424,10
T-test,10.577730,79,two-sided,8.502019e-17,"[16.17, 23.67]",1.182626,7.161e+13,1.000000,11


In [27]:
df_2 = pd.DataFrame()
Month_List =temp.Month.unique().tolist()
for i in Month_List:
  Leader = temp[temp.Month==i]['Leader']
  Laggard = temp[temp.Month==i]['Laggard']
  t_test = pg.ttest(Leader, Laggard,paired=False)
  t_test['Month']=i
  df_2=df_2.append(t_test)

In [28]:
df_2

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power,Month
T-test,-0.325796,181.612804,two-sided,0.744954,"[-2.33, 1.67]",0.047307,0.169,0.061660,1
T-test,-0.732099,174.681508,two-sided,0.465089,"[-2.33, 1.07]",0.108368,0.209,0.110713,2
T-test,-2.048337,169.833295,two-sided,0.042066,"[-8.89, -0.16]",0.308344,1.127,0.534339,3
T-test,-0.661196,175.068621,two-sided,0.509356,"[-6.53, 3.25]",0.097298,0.2,0.098533,4
T-test,-2.088224,165.371256,two-sided,0.038310,"[-4.78, -0.13]",0.317428,1.223,0.549890,5
T-test,0.545015,175.418383,two-sided,0.586436,"[-1.58, 2.78]",0.081662,0.187,0.084178,6
T-test,-0.947150,177.585603,two-sided,0.344850,"[-3.39, 1.19]",0.140205,0.245,0.155058,7
T-test,1.625562,174.270778,two-sided,0.105850,"[-0.35, 3.65]",0.243637,0.553,0.364381,9
T-test,0.046014,166.667991,two-sided,0.963355,"[-2.19, 2.3]",0.006979,0.165,0.050239,10
T-test,1.561046,139.158291,two-sided,0.120784,"[-0.93, 7.92]",0.242098,0.501,0.364267,11


# Overall test

In [29]:
Leader =temp['Leader']
Laggard =temp['Laggard']

In [30]:
t_test = pg.ttest(Leader, 0)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,2.861668,819,two-sided,0.004322,"[0.52, 2.81]",0.099934,2.278,0.8155


In [31]:
t_test = pg.ttest(Leader, Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,-0.958193,1719.770392,two-sided,0.3381,"[-2.27, 0.78]",0.045574,0.084,0.160456
